# Content Design for RAG
This notebook is part of a collection of material related to content design principles for retrieval-augmented generation (RAG).

You can explore the complete collection here: [Content Design for RAG on GitHub](https://github.com/spackows/ICAAI-2024_RAG-CD/blob/main/README.md)

**Example scenario**

Imagine your company sells seeds and gardening supplies online.  On your website, you have articles with gardening information and advice.  You are building a RAG solution for your company website that can answer customer questions about your products, using your website articles as a knowledge base.

# Classify input
When users submit input to your RAG solution, you need to classify the type of input so your solution can respond appropriately.

This sample notebook demonstrates a simple approach this problem.  A large language model (LLM) is a good tool for this job.

**Contents**
1. Write prompt text
2. Prompt an LLM
3. Test input

## 1. Write prompt text

The following prompt template works with many LLMs:
- The prompt instructs the LLM to classify given input
- Classes named and short descriptions are included in the prompt
- A few examples are included in the prompt (aka "few-shot prompt")
- The `%s` is a placeholder where the run-time user input will go

In [56]:
g_template_1 = """Classify the user input into one of the following classes: question, problem, instruction, keywords

Class: question
Description: The user is asking a question, like: "What time is it" or "How do you get to the airport?"

Class: problem
Description: The user is describing a problem they are having, like: "My car won't start" or "I'm getting an error when I log in"

Class: instruction
Description: The user input is an imperative statement, instructing you to perform an action.

Class: keywords
Description: The input is not a sentence, just a list of words - as for a keyword search

Examples:

User input: How large are sunflowers?
question

User input: My peppers aren't changing color, even though it's been months
problem

User input: Tell me your secrets
instruction

User input: When is the best time to water plants
question

User input: tomato varieties
keywords

User input: Bugs are eating my strawberries!
problem

User input: %s
"""

## 2. Prompt an LLM

See: [Foundation models Python library](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html)

### Prerequisites
Before you can prompt a foundation model in watsonx.ai, you must perform the following setup tasks:
- 2.1 Create an instance of the Watson Machine Learning service
- 2.2 Associate the Watson Machine Learning instance with the current project
- 2.3 Create an IBM Cloud API key
- 2.4 Look up the current project ID


#### 2.1 Create an instance of the Watson Machine Learning service
If you don't already have an instance of the IBM Watson Machine Learning service, you can create an instance of the service from the IBM Cloud catalog: [Watson Machine Learning service](https://cloud.ibm.com/catalog/services/watson-machine-learning)

#### 2.2 Associate an instance of the Watson Machine Learning service with the current project
The current project is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: [Adding associated services to a project](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp)

#### 2.3 Create an IBM Cloud API key
Create an IBM Cloud API key by following these instruction: [Creating an IBM Cloud API key](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key)

Then paste your new IBM Cloud API key in the code cell below.

In [50]:
cloud_apikey = ""

g_wml_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : cloud_apikey
}

#### 2.4 Look up the current project ID
The current project is the project in which you are running this notebook. You can get the ID of the current project programmatically by running the following cell.

In [51]:
import os

g_project_id = os.environ["PROJECT_ID"]

Just FYI: List supported models

In [52]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

model_ids = list( map( lambda e: e.value, ModelTypes._member_map_.values() ) )
model_ids

['google/flan-t5-xxl',
 'google/flan-ul2',
 'bigscience/mt0-xxl',
 'eleutherai/gpt-neox-20b',
 'ibm/mpt-7b-instruct2',
 'bigcode/starcoder',
 'meta-llama/llama-2-70b-chat',
 'meta-llama/llama-2-13b-chat',
 'ibm/granite-13b-instruct-v1',
 'ibm/granite-13b-chat-v1',
 'google/flan-t5-xl',
 'ibm/granite-13b-chat-v2',
 'ibm/granite-13b-instruct-v2',
 'elyza/elyza-japanese-llama-2-7b-instruct',
 'ibm-mistralai/mixtral-8x7b-instruct-v01-q',
 'codellama/codellama-34b-instruct-hf',
 'ibm/granite-20b-multilingual']

Now prompt an LLM ...

In [53]:
from ibm_watson_machine_learning.foundation_models import Model
import json

g_model_id = "google/flan-t5-xxl"

g_prompt_parameters = {
    "decoding_method" : "greedy",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 20
}

def classify( prompt_template, input_txt, b_debug=False ):
    model = Model( g_model_id, g_wml_credentials, g_prompt_parameters, g_project_id )
    prompt_text = prompt_template % input_txt
    raw_response = model.generate( prompt_text )
    if b_debug:
        print( "prompt_text:\n'" + prompt_text + "'\n" )
        print( "raw_response:\n" + json.dumps( raw_response, indent=3 ) )
    if ( "results" in raw_response ) \
       and ( len( raw_response["results"] ) > 0 ) \
       and ( "generated_text" in raw_response["results"][0] ):
        return raw_response["results"][0]["generated_text"]
    else:
        return ""

In [70]:
txt = "how large a pot do I need for growing peppers"
input_type = classify( g_template_1, txt )
print( "Input type: " + input_type )

Input type: question


In [69]:
txt = "give me a discount"
input_type = classify( g_template_1, txt )
print( "Input type: " + input_type )

Input type: instruction


In [68]:
txt = "only one cucmber got ripe and then there were no more"

input_type = classify( g_template_1, txt )
print( "Input type: " + input_type )

Input type: problem


In [67]:
txt = "seeds for sale"
input_type = classify( g_template_1, txt )
print( "Input type: " + input_type )

Input type: keywords


For questions, it's useful to know what kind of question it is.

In [60]:
g_template_2 = """Classify the given question into one of the following classes: what-is, how-to

Class: what-is
Description: The question is asking for factual details or for an explanation of a concept or idea

Class: how-to
Description: The question is asking for instructions on how to do something

Examples:

User input: How large are sunflowers
what-is

User input: What's the best way to water plants?
how-to

User input: How can you tell when pepper are ripe?
how-to

User input: How do you transplant a seedling
how-to

User input: What kinds of pests bother tomoatoes?
what-is

User input: %s
"""

In [71]:
txt = "what is tomato blight?"
input_type = classify( g_template_2, txt )
print( "Question type: " + input_type )

Question type: what-is


In [72]:
txt = "What is the best way to prune blueberry bushes"

input_type = classify( g_template_2, txt )
print( "Question type: " + input_type )

Question type: how-to
